In [6]:
import gradio as gr
import pickle

In [7]:
with open(f"./model/model.pkl", "rb") as f:
    model = pickle.load(f)

In [8]:
def get_yes_no_question(yes_no):
    yes_no_mapping = {
        "No" : 0,
        "Yes" : 1
    }
    return yes_no_mapping.get(yes_no, -1)

def get_gender(gender):
    gender_mapping = {
        "Female" : 0,
        "Male" : 1
    }
    return gender_mapping.get(gender, -1)

status = {
    0 : "Dropout",
    1 : "Graduate"
}

def predict_status(_sem_enrolled, _scholarship_holder, _sem_approved, _sem_credited, 
                _tuition_fees, _sem_evaluations, _gender, _debt):
    scholarship_holder = get_yes_no_question(_scholarship_holder)
    tuition_fees = get_yes_no_question(_tuition_fees)
    gender = get_gender(_gender)
    debt = get_yes_no_question(_debt)
    data = [[debt, _sem_approved, _sem_evaluations, _sem_credited, 
             _sem_enrolled, scholarship_holder, 
             tuition_fees, gender]]
    prediction = model.predict(data)[0]
    prediction_proba = model.predict_proba(data)[0][prediction]
    if prediction == 0:
        return f"The student might {status[prediction]}, model confidence is {prediction_proba:.2f}"
    if prediction == 1:
        return f"The student should {status[prediction]}, model confidence is {prediction_proba:.2f}"

In [9]:
with gr.Blocks(title="Student Status Prediction") as demo:
    gr.Markdown("""
                # 🎒 Student Status Prediction
                # Dicoding - Solving Educational Institution Problem
                ## Made by : Muhammad Hafizh Dzaki
                ## Gihub Repo : [Here](https://github.com/hfzdzakii/Dicoding-SolvingEducationIntsituteProblem)
                """)
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Input Variables")
            sem_approved = gr.Number(label="Sum of 2nd Semester Curricular Units Approved:",
                                     value=0, minimum=0, maximum=24)
            sem_evaluations = gr.Number(label="Sum of 2nd Semester Curricular Units Evalutions:",
                                        value=0, minimum=0)
            sem_credited = gr.Number(label="Sum of 2nd Semester Curricular Units Credited:",
                                     value=0, minimum=0, maximum=24)
            sem_enrolled = gr.Number(label="Sum of 2nd Semester Curricular Units Enrolled:",
                                     value=0, minimum=0, maximum=24)
            debt = gr.Radio(label="Having Debt?",
                            choices=["No", "Yes"], value="No")
            scholarship_holder = gr.Radio(label="Scholarship Holder?",
                                          choices=["No", "Yes"], value="No")
            tuition_fees = gr.Radio(label="Tuition Fees Payed?",
                                    choices=["No", "Yes"], value="No")
            gender = gr.Radio(label="Gender:", 
                              choices=["Male", "Female"], value="Male")
            
        with gr.Column():
            gr.Markdown("""### Example Data
                        Choose one from list below to fill input immediately!
                        """)
            gr.Examples(
                examples=[
                    [6, "No", 5, 0, "Yes", 13, "Female", "Yes"],
                    [5, "No", 0, 0, "Yes", 0, "Male", "No"],
                    [7, "No", 6, 2, "Yes", 10, "Female", "Yes"],
                    [5, "Yes", 3, 0, "No", 9, "Female", "No"],
                    [6, "Yes", 6, 0, "No", 6, 'Female', "Yes"],
                    [6, "Yes", 6, 2, "No", 6, "Female", "No"]
                ],
                inputs=[sem_enrolled, scholarship_holder, sem_approved,
                         sem_credited, tuition_fees, sem_evaluations, gender,
                         debt]
            )
            
            gr.Markdown("### Predict and Result")
            predict_button = gr.Button("Predict", variant="primary")
            prediction = gr.Textbox(label="Prediction", interactive=False)
            
    predict_button.click(
        fn=predict_status,
        inputs=[sem_enrolled, scholarship_holder,
                sem_approved,
                sem_credited, tuition_fees,
                sem_evaluations, gender, debt],
        outputs=prediction,
    )

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
